In [0]:
with cte_data (id, x, y, xy, pt) as
(
select monotonically_increasing_id()
     , split_part(value, ',', 1)::int as x
     , split_part(value, ',', 2)::int as y
     , x + y / 100000::decimal(14,7)
     , ST_Point(x, y)
  from read_files('/Volumes/waldar/fabien/aoc2025/aoc2025_09_input.txt', format => 'text', wholeText => false) as t (value)
/*from values ('7,1')
            , ('11,1')
            , ('11,7')
            , ('9,7')
            , ('9,5')
            , ('2,5')
            , ('2,3')
            , ('7,3') as t (value)*/
)
  ,  cte_polygon (arg, poly) as
(
select array_sort(array_agg(struct(id, pt)), (l, r) -> (l.id > r.id)::int * 2 - 1) as arg
     , st_makepolygon(st_makeline(array_append(arg.pt, arg[0].pt)))
  from cte_data
)
-- This approach is more performant because computing the st_contains is where most of the time geos
-- Running the operation on the sorted array make it stops computing as soon as the largest area is found.
select max((abs(t2.x - t1.x) + 1l)::bigint * (abs(t2.y - t1.y)::bigint + 1l)) as part1
     , aggregate( array_sort( array_agg(struct((abs(t2.x - t1.x) + 1l)::bigint * (abs(t2.y - t1.y)::bigint + 1l) as a, array(t1.pt, t2.pt) as pts))
                            , (l, r) -> (l.a < r.a)::int * 2 - 1)
                , null::bigint
                , (agg, p) -> coalesce(agg, case when st_contains(any_value(poly), st_envelope(st_makeline(p.pts))) then p.a end)
                ) as part2
  from cte_data as t1
  join cte_data as t2 on t1.xy < t2.xy
  join cte_polygon    on true;


select filter( array_sort( array_agg(struct((abs(t2.x - t1.x) + 1l)::bigint * (abs(t2.y - t1.y)::bigint + 1l) as a, array(t1.pt, t2.pt) as pts))
                         , (l, r) -> (l.a < r.a)::int * 2 - 1)
             , p -> st_contains(any_value(poly), st_envelope(st_makeline(p.pts))))[0].a
  from cte_data as t1
  join cte_data as t2 on t1.xy < t2.xy
  join cte_polygon    on true;






  ,  cte_prep (area, incl) as
(
select (abs(t2.x - t1.x) + 1l)::bigint * (abs(t2.y - t1.y)::bigint + 1l)
     , st_contains(poly, st_envelope(st_makeline(array(t1.pt, t2.pt))))
  from cte_data as t1
  join cte_data as t2 on (t1.x, t1.y) < (t2.x, t2.y)
  join cte_polygon    on true
)
select max(area) as part1
     , max(area) filter(where incl) as part2
  from cte_prep;

4767418746; 1461987144;
;



with cte_data (x, y, xy, pt) as
(
select split_part(value, ',', 1)::int as x
     , split_part(value, ',', 2)::int as y
     , x + y / 1000000
     , ST_Point(x, y)
--from read_files('/Volumes/waldar/fabien/aoc2025/aoc2025_09_input.txt', format => 'text', wholeText => false) as t (value)
  from values ('7,1')
            , ('11,1')
            , ('11,7')
            , ('9,7')
            , ('9,5')
            , ('2,5')
            , ('2,3')
            , ('7,3') as t (value)
)
  ,  cte_polygon (arg, poly, arg2, poly2) as
(
select array_agg(pt) as arg
     , st_makepolygon(st_makeline(array_append(arg, arg[0])))
     , array_agg(struct(x, y)) as arg2
     , array_append(arg2, arg2[0])
  from cte_data
)
select *
  from cte_polygon;


WITH cte_data (x, y, xy, idx) AS (
  SELECT split_part(value, ',', 1)::int AS x,
         split_part(value, ',', 2)::int AS y,
         x + y / 1000000.0 AS xy,
         monotonically_increasing_id() AS idx
  FROM read_files('/Volumes/waldar/fabien/aoc2025/aoc2025_09_input.txt', format => 'text', wholeText => false) as t (value)
  -- VALUES ('7,1'), ('11,1'), ('11,7'), ('9,7'), ('9,5'), ('2,5'), ('2,3'), ('7,3') AS t(value)
),

cte_edges (idx, x1, y1, x2, y2) AS (
  SELECT d1.idx,
         d1.x AS x1, d1.y AS y1,
         COALESCE(d2.x, min(case d1.idx when 1 then d1.x end) over()) AS x2,
         COALESCE(d2.y, min(case d1.idx when 1 then d1.y end) over()) AS y2
  FROM cte_data d1
  LEFT JOIN cte_data d2 ON d2.idx = d1.idx + 1
)
,

cte_rects AS (
  SELECT t1.xy AS xy1, t2.xy AS xy2,
         LEAST(t1.x, t2.x) AS min_x, GREATEST(t1.x, t2.x) AS max_x,
         LEAST(t1.y, t2.y) AS min_y, GREATEST(t1.y, t2.y) AS max_y,
         (ABS(t2.x - t1.x) + 1)::bigint * (ABS(t2.y - t1.y) + 1)::bigint AS area
  FROM cte_data t1
  JOIN cte_data t2 ON t1.xy < t2.xy
),

cte_checks AS (
  SELECT 
    r.xy1, r.xy2, r.area,
    
    -- Ray casting for all 4 corners
    SUM(CASE WHEN ((e.y1 > r.min_y) != (e.y2 > r.min_y)) 
              AND r.min_x < e.x1 + (r.min_y - e.y1) * (e.x2 - e.x1) * 1.0 / NULLIF(e.y2 - e.y1, 0) 
         THEN 1 ELSE 0 END) % 2 AS c1_in,
    SUM(CASE WHEN ((e.y1 > r.min_y) != (e.y2 > r.min_y)) 
              AND r.max_x < e.x1 + (r.min_y - e.y1) * (e.x2 - e.x1) * 1.0 / NULLIF(e.y2 - e.y1, 0) 
         THEN 1 ELSE 0 END) % 2 AS c2_in,
    SUM(CASE WHEN ((e.y1 > r.max_y) != (e.y2 > r.max_y)) 
              AND r.min_x < e.x1 + (r.max_y - e.y1) * (e.x2 - e.x1) * 1.0 / NULLIF(e.y2 - e.y1, 0) 
         THEN 1 ELSE 0 END) % 2 AS c3_in,
    SUM(CASE WHEN ((e.y1 > r.max_y) != (e.y2 > r.max_y)) 
              AND r.max_x < e.x1 + (r.max_y - e.y1) * (e.x2 - e.x1) * 1.0 / NULLIF(e.y2 - e.y1, 0) 
         THEN 1 ELSE 0 END) % 2 AS c4_in,
    
    -- Proper segment-rectangle intersection: check if edge crosses any of the 4 rectangle sides
    MAX(CASE 
      -- Edge crosses bottom side (y = min_y, strictly between endpoints)
      WHEN (e.y1 - r.min_y) * (e.y2 - r.min_y) < 0 
       AND e.x1 + (r.min_y - e.y1) * (e.x2 - e.x1) * 1.0 / (e.y2 - e.y1) > r.min_x
       AND e.x1 + (r.min_y - e.y1) * (e.x2 - e.x1) * 1.0 / (e.y2 - e.y1) < r.max_x
      THEN 1
      -- Edge crosses top side (y = max_y)
      WHEN (e.y1 - r.max_y) * (e.y2 - r.max_y) < 0 
       AND e.x1 + (r.max_y - e.y1) * (e.x2 - e.x1) * 1.0 / (e.y2 - e.y1) > r.min_x
       AND e.x1 + (r.max_y - e.y1) * (e.x2 - e.x1) * 1.0 / (e.y2 - e.y1) < r.max_x
      THEN 1
      -- Edge crosses left side (x = min_x)
      WHEN (e.x1 - r.min_x) * (e.x2 - r.min_x) < 0 
       AND e.y1 + (r.min_x - e.x1) * (e.y2 - e.y1) * 1.0 / (e.x2 - e.x1) > r.min_y
       AND e.y1 + (r.min_x - e.x1) * (e.y2 - e.y1) * 1.0 / (e.x2 - e.x1) < r.max_y
      THEN 1
      -- Edge crosses right side (x = max_x)
      WHEN (e.x1 - r.max_x) * (e.x2 - r.max_x) < 0 
       AND e.y1 + (r.max_x - e.x1) * (e.y2 - e.y1) * 1.0 / (e.x2 - e.x1) > r.min_y
       AND e.y1 + (r.max_x - e.x1) * (e.y2 - e.y1) * 1.0 / (e.x2 - e.x1) < r.max_y
      THEN 1
      ELSE 0
    END) AS has_cross
        
  FROM cte_rects r
  CROSS JOIN cte_edges e
  GROUP BY r.xy1, r.xy2, r.area
)
SELECT  MAX(area) AS part1,
       MAX(area) FILTER (WHERE c1_in = 1 AND c2_in = 1 AND c3_in = 1 AND c4_in = 1 AND has_cross = 0) AS part2
FROM cte_checks
;

